In [1]:
import numpy as np
import csv
import pandas as pd
import tensorflow as tf

In [2]:
## 读取csv文件
data_train_origin = []
train = csv.reader(open('train.tsv', 'rt'))
for row in train:
    data_train_origin.append(row)
    
data_test_origin = []
train = csv.reader(open('test.tsv', 'rt'))
for row in train:
    data_test_origin.append(row)

In [4]:
## 数据的分割
data_train = []
for item in data_train_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_train.append(item[0].split('\t'))
    
data_test = []
for item in data_test_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_test.append(item[0].split('\t'))

In [5]:
## 删除标题
data_train.remove(data_train[0])
data_test.remove(data_test[0])

In [6]:
## 简单预处理
for i in range(len(data_train)):
    data_train[i][0] = int(data_train[i][0])
    data_train[i][1] = int(data_train[i][1])
    data_train[i][2] = data_train[i][2].replace('  ',' ')
    data_train[i][3] = int(data_train[i][3])
    
for i in range(len(data_test)):
    data_test[i][0] = int(data_test[i][0])
    data_test[i][1] = int(data_test[i][1])
    data_test[i][2] = data_test[i][2].replace('  ',' ')

In [7]:
## 切分单词
tokenized_lines = []
for item in data_train:
    tokenized_lines.append(item[2].split(' '))

In [8]:
## 去除标点，替换大小写
punctuation = [',', ':', ';', '.', "'", '"', "’", '?', '/', '-', '+', '&', '(', ')']
clean_tokenized = []
for item in tokenized_lines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, " ")
        tokens.append(token)
    clean_tokenized.append(tokens)

In [9]:
## 生成原始的词汇集合,剔除了只出现过一次的token
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)

In [10]:
## 加入起止符号和OOV
unique_tokens.append('_UNK')
unique_tokens.append('_PAD')

In [11]:
## 对每个tokens进行编号
dic = {}
for i in range(len(unique_tokens)):
    dic[unique_tokens[i]]=i

In [12]:
## 计算每个句子的长度并存储
lens = []
for words in clean_tokenized:
    lens.append(len(words))
print(max(lens))

36


In [13]:
## 将每个句子表征称为index的形式，并补全到最大长度
representations = []
for words in clean_tokenized:
    reps = []
    if len(words) < max(lens):
        for i in range(max(lens)-len(words)):
            words.append('_PAD')       
    for word in words:
        if (word in unique_tokens):
            reps.append(dic[word])
        else:
            reps.append(dic['_UNK'])
    representations.append(reps)

In [14]:
## 定义x，y
x = representations
y = []
for item in data_train:
    y.append(item[3])

In [15]:
## y生成one-hot vector
classes = np.max(y) + 1
y = np.eye(classes)[y]

In [16]:
## 划分集合
n_train = int(len(x)*0.8)
n_test = int(len(x)*0.1)
n_dev = int(len(x) - n_train -n_test)

import random
index_train = random.sample(range(len(x)),n_train)
index_dev = random.sample(list(set(range(len(x)))^set(index_train)) ,n_dev)
index_test = list(set(list(set(range(len(x)))^set(index_train)))^set(index_dev)) 

x_train = []
y_train = []
lens_train = []
for index in index_train:
    x_train.append(x[index])
    y_train.append(y[index])
    lens_train.append(lens[index])

x_dev = []
y_dev = []
lens_dev = []
for index in index_dev:
    x_dev.append(x[index])
    y_dev.append(y[index])
    lens_dev.append(lens[index])

x_test = []
y_test = []
lens_test = []
for index in index_test:
    x_test.append(x[index])
    y_test.append(y[index])
    lens_test.append(lens[index])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_dev = np.array(x_dev)
y_dev = np.array(y_dev)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [17]:
## 定义一个随机函数，随机抽取
import random
def random_sample(x,y,lens_):
    index = random.sample(range(len(x)),1)[0]
    balanced_m = np.concatenate((np.ones((lens_[index], 5),dtype = np.float32),np.zeros((max(lens)-lens_[index], 5),dtype = np.float32)))
    return x[index],y[index],balanced_m

用于随机梯度下降法

In [ ]:
## 通过tf构建logistic model
# 定义输入和输出,以及一个平衡矩阵
x_input = tf.placeholder(tf.int32,[max(lens)])
y_label = tf.placeholder(tf.float32,[5])
balanced_matrix = tf.placeholder(tf.float32,[max(lens),5])

平衡矩阵是用来补偿padding所带来的影响，通过对于padding单词计算损失函数时全0处理来减小它们在训练时的作用

In [19]:
# 定义word embedding层
lookup = tf.Variable(tf.random_normal([len(unique_tokens),128],stddev=1))

In [20]:
# 定义参数
W = tf.Variable(tf.random_normal([128,5],stddev=1))
b = tf.Variable(tf.random_normal([1,5],stddev=1))

In [21]:
# 计算embedding lookup
x_embedding = tf.nn.embedding_lookup(lookup,x_input)

实现embedding lookup的代码：可以通过tf.nn.embedding_lookup(lookup matrix, input_index)实现
难点：tensor不能迭代，因此先通过一次session计算出x，再通过迭代计算每条句子所表征的矩阵
使用随机梯度下降法，每次学习一个样本，可以避免这个问题

In [22]:
# 计算预测值y_,通过对每个词的预测求平均获得对于整个文本的情感分析
y_pre = tf.reduce_mean(tf.nn.sigmoid(tf.matmul(x_embedding,W)+b)*balanced_matrix,axis=0)

对于每个embedding的输入，通过sigmoid函数计算预测向量. PS:
1、 当一个句子文本长度小于最大长度时，需要对去除padding部分对于预测的影响，通过构造一个只包含0，1的矩阵，当行数小于文本长度的时候，值为1；当大于文本长度但小于最大长度时，值为0，再将两个矩阵对应相乘（不是点积）得到新的矩阵。实现方法：np.ones(shape,dtype), np.zeros(shape,dtype), np.concatenate((a,b))实现。
2、 对于矩阵表示一个文本中每个词的情感情况，通过求平均值来表征句子的情感分类。实现方法：tf.reduce_mean()通过设置axis参数来调整形状。

In [23]:
# 计算loss function
L = -tf.reduce_mean(y_label*tf.log(y_pre) + (1-y_label)*tf.log(1-y_pre))

In [24]:
# 梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.000001).minimize(L)

In [25]:
# 初始化variables
init = tf.global_variables_initializer()

In [ ]:
# 进行训练
L_train = []
L_dev = []
n = 0
m = 0

with tf.Session() as sess:
    sess.run(init)
    while (n < 20 and m< 10000):
        x_data,y_data,balanced_m = random_sample(x_train,y_train,lens_train)      
        x_dev_data,y_dev_data,balanced_dev_m = random_sample(x_dev,y_dev,lens_dev)

        sess.run(train_step,feed_dict={x_input:x_data,y_label:y_data,balanced_matrix:balanced_m})
        L_dev.append(sess.run(L,feed_dict={x_input:x_dev_data,y_label:y_dev_data,balanced_matrix:balanced_dev_m}))
        L_train.append(sess.run(L,feed_dict={x_input:x_data,y_label:y_data,balanced_matrix:balanced_m}))
        m += 1
        if (len(L_dev)>=2):
            if (L_dev[-1] > L_dev[-2]):
                n += 1
            else:
                n = 0
        print (str(m)+' iteration: ',n)
        
    W_ = sess.run(W)
    b_ = sess.run(b)
    lookup_ = sess.run(lookup)
    y_test_pre = []
    for j in range(len(x_test)):
        balanced_test_m = np.concatenate((np.ones((lens_test[j], 5),dtype = np.float32),np.zeros((max(lens)-lens_test[j], 5),dtype = np.float32)))
        y_test_pre.append(sess.run(y_pre,feed_dict={x_input:x_test[j],y_label:y_test[j],balanced_matrix:balanced_test_m}))

SDG：记录每次迭代后的train set和dev set的L值，直到dev set的L值不再减小或者迭代10000次为止

In [27]:
## 设计模型在test set上的验证
def evaluate(y_test_pre,y_test):    
    y = []
    y_ = []
    for onehot in y_test:
        y.append(np.where(onehot > 0.5)[0][0])    
    for vector in y_test_pre:
        y_.append(np.where(vector == max(vector))[0][0])
    y = np.array(y)
    y_ = np.array(y_)
    loss = -np.average(y_test*np.log(y_test_pre)+(1-np.array(y_test))*np.log(1-np.array(y_test_pre)))
    accuracy = float(np.sum(np.array(y)==np.array(y_)))/float(len(y))
    precision = []
    recall = []
    f1 = []
    elems = list(set(y))
    for elem in elems:
        p = 0
        r = 0
        n = 0
        for i in range(len(y)):
            if (y[i] == elem):
                p += 1
            if (y_[i] == elem):
                r += 1
            if (y[i] == y_[i] and y[i] == elem):
                n +=1
        if (p != 0):
            precision.append(float(n)/float(p))
        else:
            precision.append(0)
        if (r != 0):
            recall.append(float(n)/float(r))
        else:
            recall.append(0)
    for j in range(len(precision)):
        if (precision[j]!=0 or recall[j] != 0):
            f1.append(2*precision[j]*recall[j]/(precision[j]+recall[j]))
        else:
            f1.append(0)
    return loss,accuracy,elems,precision,recall,f1

In [28]:
# evaluate
loss,accuracy,elems,precision,recall,f1 = evaluate(y_test_pre,y_test)

In [29]:
loss

1.0589316942770737

In [30]:
accuracy

0.2653061224489796

In [31]:
precision

[1.0, 0.42857142857142855, 0.1724137931034483, 0.3333333333333333, 0.0]

In [32]:
recall

[0.2222222222222222, 0.2, 0.7142857142857143, 0.375, 0.0]

In [33]:
f1

[0.3636363636363636,
 0.27272727272727276,
 0.2777777777777778,
 0.35294117647058826,
 0]